## Setup

In [ ]:
!pip install pandas numpy scikit-learn lightgbm transformers

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
from lightgbm import LGBMClassifier
from transformers import AutoTokenizer, AutoModel
from sklearn.preprocessing import LabelEncoder

In [ ]:
def load_data(train_path='data/train.csv', test_path='data/test.csv'):
    train_data = pd.read_csv(train_path)
    test_data = pd.read_csv(test_path)
    return train_data, test_data

train_data, test_data = load_data()
print(train_data.head())